In [2]:
import matplotlib
#%matplotlib tk
%matplotlib inline
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio

%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import matplotlib.cm as cm
import numpy as np
import os
from tqdm import tqdm

#
import sys
sys.path.append('/home/cat/code/gerbil/')

# 
from utils.track import track
from utils.convert import convert
from utils.ethogram import ethogram
from utils.cohort import cohort
from utils.cohort.cohort import CohortProcessor
from utils.ethogram import ethogram

Autosaving every 180 seconds


In [3]:
######################################################
############# LOAD ANIMAL DATABASE ###################
######################################################
fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/database.xlsx'
#fname_spreadsheet = '/media/cat/4TBSSD/dan/cohort2/P21_P28/database.xlsx'

#
cohort = cohort.CohortProcessor(fname_spreadsheet)
#cohort.fname_spreadsheet = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort2/database.xlsx'
cohort.load_database()

#
cohort.ethogram = ethogram

# print (list(zip(cohort.PDays,cohort.Start_times))[0])
# print (list(zip(cohort.PDays,cohort.Start_times))[1])
# print (list(zip(cohort.PDays,cohort.Start_times))[100])

DF:     Unnamed: 0  Cohort NN Type                                   Filename Dev Day  # of Gerbils Time of Light Switch  Shaved?  Predicted?                                     Slp filename Start time
0           0       3     Day  2020_08_23_16_00_38_413695_compressed.mp4     P15             6                  NaN        1           1    2020_08_23_16_00_38_413695_compressed_Day.slp   16:00:38
1           1       3     Day  2020_08_23_17_06_55_680620_compressed.mp4     P15             6                  NaN        1           1    2020_08_23_17_06_55_680620_compressed_Day.slp   17:06:55
2           2       3     Day  2020_08_23_18_13_11_710750_compressed.mp4     P15             6                  NaN        1           1    2020_08_23_18_13_11_710750_compressed_Day.slp   18:13:11
3           3       3    Both  2020_08_23_19_19_27_606024_compressed.mp4     P15             6    47:43, day, night        1           1    2020_08_23_19_19_27_606024_compressed_Day.slp   19:19:27
4         

In [4]:
#################################################
########### COMPUTE HUDDLE COMPOSITION ##########
#################################################

#
cohort.video_frame_rate = 24
cohort.parallel_flag = True

# set the flags for the huddle corrections
cohort.fix_track_flag = True
cohort.interpolate_flag = True
cohort.load_huddle_tracks()

# # this just loads the _spine.npy files;
cohort.use_nohuddle = True
cohort.load_feature_tracks()

# # this computes mode-based occupancy in the nest with 1min bins
cohort.median_filter_width = cohort.video_frame_rate*5+1 # 1: there is no smotthing; n x frame rate gives you n seconds of somothing (always add 1 to get odd number)
#cohort.n_frames_per_bin = cohort.video_frame_rate  # single second resolution
cohort.n_frames_per_bin = cohort.video_frame_rate #*60  # single second resolution
cohort.n_cores = 24
cohort.compute_huddle_composition()
print ("# of huddle composition arrays: ", len(cohort.huddle_comps_binned), cohort.huddle_comps_binned[0].shape)


 # of files:  352 example:  /mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/cohort3/huddles/2020_08_23_16_00_38_413695_compressed_Day_huddle_spine_fixed_interpolated.npy
# of feature tracks:  352 , missing:  0


100%|█████████████████████████████████████████| 352/352 [02:51<00:00,  2.05it/s]

# of huddle composition arrays:  352 (6, 3601)


In [10]:
#################################################
########### DISPLAY HUDDLE COMPOSITION ##########
#################################################
%matplotlib qt

#
cohort.forward_padding = 20 # padding in percent
cohort.generate_huddle_composition_ethograms()

#
cohort.show_huddle_composition_ethogram_all_animals()

#
#cohort.show_huddle_composition_ethogram_full_huddle()

#
animal_ids = np.arange(6)
# cohort.show_huddle_composition_ethogram_specific_animals(animal_ids)


size of img:  (86400, 96)  size of flatten image:  (1382400, 6)


100%|███████████████████████████████████████| 352/352 [00:00<00:00, 1070.98it/s]


In [14]:
np.save('/home/cat/Downloads/cohort3_huddle_comp.npy', cohort.huddle_ethogram)

In [19]:
cohort.show_combined_ethogram()

In [104]:
##########################################################
########### COMPUTE FLATTENED VERSION OF ETHOGRAM ########
##########################################################

# OPTIONAL
# print (cohort.huddle_ethogram.shape)

# # flatten the ethogram
# ethogram_flat = []

# for k in range(0,cohort.huddle_ethogram.shape[1],6):
#     ethogram_flat.append(cohort.huddle_ethogram[:,k:k+6])
# ethogram_flat = np.vstack(ethogram_flat).T
# print (ethogram_flat.shape)

# # optional visualize
# if False:
#     plt.figure()

#     plt.imshow(ethogram_flat, aspect='auto', 
#                interpolation='none',  cmap='jet')
#     plt.show()

(86400, 96)
(6, 1382400)


In [16]:
##############################################################################
########### COMPUTE FLATTENED VERSION OF ETHOGRAM + CORRELATION ARRAY ########
##############################################################################

# FIGURE 1: CORRELOGRAMS <- not super informative!
# flatten the ethogram
ethogram_flat = []

for k in range(0,cohort.huddle_ethogram.shape[1],6):
    ethogram_flat.append(cohort.huddle_ethogram[:,k:k+6])
ethogram_flat = np.vstack(ethogram_flat).T
print (ethogram_flat.shape)

# compute the correlograms between each pair of animals
from scipy.stats import pearsonr

#
window = 100
t = np.arange(-window,window,1)
from tqdm import trange 

#
cc_array = []
plt.figure()
for k in range(6):
    for p in trange(k+1,6,1):
        cc = []
        for z in range(-window, window,1):
            cc.append(pearsonr(np.roll(ethogram_flat[k],z), 
                                    ethogram_flat[p])[0])
        cc_array.append([k,p,cc])

        #
        ax=plt.subplot(6,6,k*6+p+1)
        plt.plot(t,cc)
        plt.plot([0,0],[-1,1],'--',c='black')
#

plt.show()

(6, 1382400)


100%|█████████████████████████████████████████████| 1/1 [00:15<00:00, 15.53s/it]
0it [00:00, ?it/s]


In [20]:
####################################################
########### SELECT DAY OF ETHOGRAM TO ANALYZ #######
####################################################

print ("# animals, # seconds: ", ethogram_flat.shape)

# here P15 is day 0 for clarity
day_start = 0
day_end = 16

#
times = np.arange(day_start*24*60*60,
                  day_end*24*60*60)
print ("times: ", times)

#
ethogram_flat_selected = ethogram_flat[:,times].copy()
print (ethogram_flat_selected.shape)




# animals, # seconds:  (6, 1382400)
times:  [      0       1       2 ... 1382397 1382398 1382399]
(6, 1382400)


In [21]:
####################################################
########### COMPUTE PAIR WISE SCATTER PLOTS ########
####################################################

%matplotlib qt

plt.figure(figsize=(6,6))
clrs = ['red','blue','green','black','purple','orange']
clrs = ['#cb3311', '#4477aa', '#ccbb44', '#228833', '#ee7733', '#66ccee']
markers = ["," , "o" , "v" , "^" , "<", ">"]

#
follow_window = 10
xlim=0
ratios=[]
for k in trange(6):
# for k in [0]:
    # for p in range(k+1,6,1):
    for p in range(6):
    # for p in [1]:
        if k==p:
            continue
        t1 = ethogram_flat_selected[k].copy()
        idx1 = np.where(t1>0)[0]
        t1[idx1] = 1
        t2 = ethogram_flat_selected[p].copy()
        idx2 = np.where(t2>0)[0]
        t2[idx2] = 1

        # find order of events
        a1 = 0     #the number of times animal 1 precedes animal 2
        a2 = 0     # same but opposite order

        #
        ctr = 0
        while ctr<idx1.shape[0]:
        #for i in idx1:

            # check to see if there is a switch from 0
            if t2[idx1[ctr]]==0:
                # to a 1 in the next window
                if np.sum(t2[idx1[ctr]:idx1[ctr]+follow_window])>0:
                    a1+=1

                    # also advance the ctr value at least 10 sec
                    ctr0 = ctr
                    try:
                        while idx1[ctr+1]<(idx1[ctr0]+follow_window) and (ctr<idx1.shape[0]-2):
                            ctr+=1
                            continue
                    except:
                        pass
            ctr+=1

        #
        ctr = 0
        while ctr<idx2.shape[0]:
        #for i in idx1:

            # check to see if there is a switch from 0
            if t1[idx2[ctr]]==0:
                # to a 1 in the next window
                if np.sum(t1[idx2[ctr]:idx2[ctr]+follow_window])>0:
                    a2+=1

                    # also advance the ctr value at least 10 sec
                    ctr0 = ctr
                    try:
                        while idx2[ctr+1]<(idx2[ctr0]+follow_window) and (ctr<idx2.shape[0]-2):
                            ctr+=1
                            continue
                    except:
                        pass
            ctr+=1
        xlim = np.max((xlim,a1,a2))
        plt.scatter(a1,
                    a2,
                    #marker=markers[k],
                    marker=markers[p],
                    s=100,
                    #c=clrs[p],
                    c=clrs[k],

                    label=str(k)+'->'+str(p))


        # print ("animal1 : ", k, ", animal2: ", p, "# of leading behaviours: ", a1)
        # print ("animal1 : ", k, ", animal2: ", p, "# of following behaviours: ", a2)
        ratio = a1/(a1+a2)
        # print ("ratio: ", ratio)
        ratios.append([a1,a2,ratio])
        #print ("a2: ", a2)
#
plt.plot([0,xlim*1.25],[0,xlim*1.25],'--',c='black')
plt.legend(fontsize=8, ncol=6)
plt.title("# of behavior sequences (1sec resulution; 10sec follow window))")
plt.xlim(0,700) 
# plt.xlim(left=0)
plt.ylim(0,700) 
# plt.ylim(bottom=0)
plt.title("start day: "+str(day_start) + ",  day end: "+str(day_end))
plt.xlabel("Animal 1 precedes animal 2")
plt.ylabel("Animal 2 precedes animal 1")
plt.show()
# plt.savefig('/home/cat/Downloads/P26-.png')


100%|█████████████████████████████████████████████| 6/6 [02:39<00:00, 26.61s/it]


In [33]:
#########################################################################################
########### COMPUTE PAIR WISE SCATTER PLOTS - FOR SINGLE ANIMAL OVER DEVELOPMENT ########
#########################################################################################

plt.figure(figsize=(6,6))
clrs = ['red','blue','green','black','purple','orange']
clrs = ['#cb3311', '#4477aa', '#ccbb44', '#228833', '#ee7733', '#66ccee']
markers = ["," , "o" , "v" , "^" , "<", ">"]

#
follow_window = 10
xlim=0
ratios = np.empty((16,6,6))
ratios[:] = np.nan

day_starts = np.arange(17)
day_ends = np.arange(1,17,1)

for d in range(16):
    day_start = day_starts[d]
    day_end = day_ends[d]
    #
    times = np.arange(day_start*24*60*60,
                      day_end*24*60*60)
    print ("times: ", times)

    #
    ethogram_flat_selected = ethogram_flat[:,times].copy()

    #for k in trange(6):
    for k in [1]:
        #for p in range(k+1,6,1):
        for p in range(6):
        # for p in [3]:
            if k==p:
                continue
            t1 = ethogram_flat_selected[k].copy()
            idx1 = np.where(t1>0)[0]
            t1[idx1] = 1
            t2 = ethogram_flat_selected[p].copy()
            idx2 = np.where(t2>0)[0]
            t2[idx2] = 1

            # find order of events
            a1 = 0     #the number of times animal 1 precedes animal 2
            a2 = 0     # same but opposite order

            #
            ctr = 0
            while ctr<idx1.shape[0]:
            #for i in idx1:

                # check to see if there is a switch from 0
                if t2[idx1[ctr]]==0:
                    # to a 1 in the next window
                    if np.sum(t2[idx1[ctr]:idx1[ctr]+follow_window])>0:
                        a1+=1

                        # also advance the ctr value at least 10 sec
                        ctr0 = ctr
                        try:
                            while idx1[ctr+1]<(idx1[ctr0]+follow_window) and (ctr<idx1.shape[0]-2):
                                ctr+=1
                                continue
                        except:
                            pass
                ctr+=1

            #
            ctr = 0
            while ctr<idx2.shape[0]:
            #for i in idx1:

                # check to see if there is a switch from 0
                if t1[idx2[ctr]]==0:
                    # to a 1 in the next window
                    if np.sum(t1[idx2[ctr]:idx2[ctr]+follow_window])>0:
                        a2+=1

                        # also advance the ctr value at least 10 sec
                        ctr0 = ctr
                        try:
                            while idx2[ctr+1]<(idx2[ctr0]+follow_window) and (ctr<idx2.shape[0]-2):
                                ctr+=1
                                continue
                        except:
                            pass
                ctr+=1
            xlim = np.max((xlim,a1,a2))
            
            # plt.scatter(a1,
            #             a2,
            #             #marker=markers[k],
            #             marker=markers[p],
            #             # alpha = a,
            #             s=100,
            #             c=clrs[p],
            #             # c=clrs[k],
            #             label=str(k)+'->'+str(p))


            # print ("animal1 : ", k, ", animal2: ", p, "# of leading behaviours: ", a1)
            # print ("animal1 : ", k, ", animal2: ", p, "# of following behaviours: ", a2)
            ratio = a1/(a1+a2)
            # print ("ratio: ", ratio)
            # ratios.append(ratio)
            #print (d,p,k)
            ratios[d,p,k] = ratio
            #print ("a2: ", a2)
    #
# plt.plot([0,xlim*1.25],[0,xlim*1.25],'--',c='black')
# plt.legend(fontsize=8, ncol=6)
# plt.title("# of behavior sequences (1sec resulution; 10sec follow window))")
# plt.xlim(0,700) 
# # plt.xlim(left=0)
# plt.ylim(0,700) 
# # plt.ylim(bottom=0)
# plt.title("start day: "+str(day_start) + ",  day end: "+str(day_end))
# plt.xlabel("Animal 1 precedes animal 2")
# plt.ylabel("Animal 2 precedes animal 1")
# plt.show()
# plt.savefig('/home/cat/Downloads/P17.png')
print ("DONE...")
# ratios

times:  [    0     1     2 ... 86397 86398 86399]
times:  [ 86400  86401  86402 ... 172797 172798 172799]
times:  [172800 172801 172802 ... 259197 259198 259199]
times:  [259200 259201 259202 ... 345597 345598 345599]
times:  [345600 345601 345602 ... 431997 431998 431999]
times:  [432000 432001 432002 ... 518397 518398 518399]
times:  [518400 518401 518402 ... 604797 604798 604799]
times:  [604800 604801 604802 ... 691197 691198 691199]
times:  [691200 691201 691202 ... 777597 777598 777599]
times:  [777600 777601 777602 ... 863997 863998 863999]
times:  [864000 864001 864002 ... 950397 950398 950399]
times:  [ 950400  950401  950402 ... 1036797 1036798 1036799]
times:  [1036800 1036801 1036802 ... 1123197 1123198 1123199]
times:  [1123200 1123201 1123202 ... 1209597 1209598 1209599]
times:  [1209600 1209601 1209602 ... 1295997 1295998 1295999]
times:  [1296000 1296001 1296002 ... 1382397 1382398 1382399]
DONE...


In [34]:
np.save('/home/cat/Downloads/cohort3_ratios.npy', ratios)

In [35]:
ratios.shape

(16, 6, 6)

In [57]:
adults_vs_pups = np.nanmean(np.nanmean(ratios[:,2:,0:2], axis = 2), axis = 1)
adults_vs_pups = [1 - x for x in adults_vs_pups]
adults_vs_pups

[0.8539857306528456,
 0.7694813224517404,
 0.7322209906583048,
 0.7695127147796919,
 0.6046667594776838,
 0.6243599160470578,
 0.6190702561939789,
 0.6252256095932701,
 0.5593026318295464,
 0.5222075822998591,
 0.5969263611231177,
 0.5537297019231595,
 0.5609164266377038,
 0.4233959808454939,
 0.43136218757139,
 0.6021275461515856]

In [61]:
male_vs_female = ratios[:,0,1]
male_vs_female

array([0.43971631, 0.56065089, 0.49910873, 0.40378007, 0.47107438,
       0.52837327, 0.48431105, 0.46779141, 0.5988024 , 0.5597561 ,
       0.48412698, 0.52741228, 0.44275701, 0.53666667, 0.51338766,
       0.44957265])

In [62]:
%matplotlib qt

print (ratios.shape)

plt.figure()

plt.plot(adults_vs_pups, color ='black', marker = 'o', linewidth = 3, label = 'all pups')
plt.plot(male_vs_female, color ='red', marker = 'o', linewidth = 3)
    
plt.xlim(0,15)
plt.xticks(ticks = np.arange(0,16), labels = np.arange(15,31))
plt.xlabel("Postnatal day")

plt.ylim(0,1)
plt.yticks(np.arange(0,1.1,0.1))
plt.ylabel("Leader / Follower ratio")

plt.hlines(0.5,0,15, color = 'black', linestyle = 'dashdot')

plt.legend()

plt.title("Adults  vs All pups")

# plt.fill_between(np.arange(0,16), np.array([t-s for t,s in zip(test,sdt)]), np.array([t+s for t,s in zip(test,sdt)]))

plt.show()


(16, 6, 6)


Gtk-Message: 11:27:46.469: GtkDialog mapped without a transient parent. This is discouraged.
Gtk-Message: 11:28:02.455: GtkDialog mapped without a transient parent. This is discouraged.


In [9]:
%matplotlib qt

print (ratios.shape)

animals = ['female', 'male', 'pup1', 'pup2', 'pup3', 'pup4']

plt.figure()

clrs = ['#cb3311', '#4477aa', '#ccbb44', '#228833', '#ee7733', '#66ccee']

for i, a in enumerate(clrs):
    
    plt.plot(ratios[:,i,1], color = a, linewidth = 3, label = animals[i])
    
    plt.xlim(0,15)
    plt.xticks(ticks = np.arange(0,16), labels = np.arange(15,31))
    plt.xlabel("Postnatal day")
    
    plt.ylim(0,1)
    plt.yticks(np.arange(0,1.1,0.1))
    plt.ylabel("Leader / Follower ratio")
    
    plt.hlines(0.5,0,15, color = 'black', linestyle = 'dashdot')
    
    plt.legend()
    
    plt.title("Male")

plt.show()


(16, 6, 6)


Gtk-Message: 12:23:48.983: GtkDialog mapped without a transient parent. This is discouraged.

(python:5649): Gtk-WARNING **: 12:23:49.215: Failed to measure available space: The specified location is not supported
